In [8]:
%reset -f
import numpy as np
from scipy.stats import geom
import sys
from datetime import datetime
import os

In [9]:
output_log_file = "experiment_4_H.log"
log_file = open(output_log_file, 'a')
sys.stdout = log_file
print(output_log_file)
print(datetime.now().strftime("%a %b %d %H:%M:%S %Y"))

In [10]:
DEFAULT_WEIGHT_VALUE = 0  
image_mode = False
debug_mode = False

%run attack_graph_MIR100.ipynb

In [11]:
defense_rate_list = [1, 2, 3]

In [ ]:
def random_steps(route, attack_rate=None, defense_rate=None, graph=None):
    """
    Calculates probabilities for attacker movement along route.
    Returns probability distribution over possible ending nodes.
    """

    hardness = []
    for i in range(len(route) - 1):
        start_node = route[i]
        end_node = route[i + 1]
        
        weights = []
        for edge in graph[start_node][end_node].values():
            weights.append(edge.get('weight', DEFAULT_WEIGHT_VALUE))
        max_weight = max(weights) if weights else DEFAULT_WEIGHT_VALUE
        
        min_weights = []
        for edge in graph[start_node][end_node].values():
            min_weights.append(edge.get('weight', DEFAULT_WEIGHT_VALUE))
        min_weight = min(min_weights) if min_weights else DEFAULT_WEIGHT_VALUE
            
        hardness.append(np.exp(-min_weight))

    hardness = np.array(hardness)
    
    if attack_rate is None or attack_rate == 0:
        def geom_mean(x):
            positive_values = x[x > 0]
            if len(positive_values) == 0:
                return 1
            return np.exp(np.mean(np.log(positive_values)))
        
        attack_rate = 1 / geom_mean(hardness)
    
    prob = attack_rate / (attack_rate + defense_rate)
    
    x_values = np.arange(len(route))
    
    pdf_d = prob * (1-prob)**x_values
    
    pdf_d = pdf_d / np.sum(pdf_d)
    
    return pdf_d

In [13]:
%run heuristic_defense.ipynb

In [14]:
sys.stdout = sys.__stdout__
log_file.close()
with open(output_log_file, 'r') as f:
    print(f.read())